# Video-RAG using Ragie

This notebook demonstrates how to build a video-based Retrieval Augmented Generation (RAG) system. It uses [Ragie's](https://www.ragie.ai/) video ingestion and retrieval capabilities to enable semantic search and Q&A over video content.

In [1]:
import os
import time

from dotenv import load_dotenv
from ragie import Ragie

load_dotenv()

True

In [2]:
ragie = Ragie(
    auth=os.getenv('RAGIE_API_KEY')
)

## Remove previous data

In [4]:
response = ragie.documents.list()       
documents = response.result.documents

for document in documents:
    try:
        ragie.documents.delete(
            document_id=document.id
        )
        print(f"Deleted document {document.id}")
    except Exception as e:
        print(f"Error deleting document {document.id}: {e}")

## Ingestion

In [5]:
current_dir = os.path.curdir
file_path = os.path.join(current_dir, 'videos/messi-goals.mp4')
result = ragie.documents.create(request={
    "file": {
        "file_name": "messi-goals.mp4",
        "content": open(file_path, "rb"),
    },
    "mode": {
        "video": "audio_video"
    }
})

print(result)

status='partitioning' id='e80b5e34-ba44-4e88-a627-bc2199e350c2' created_at=datetime.datetime(2025, 6, 13, 10, 44, 38, 394269, tzinfo=TzInfo(UTC)) updated_at=datetime.datetime(2025, 6, 13, 10, 44, 39, 24620, tzinfo=TzInfo(UTC)) name='messi-goals.mp4' metadata={} partition='default' chunk_count=None external_id=None page_count=None


## Retrieval

In [6]:
response =  ragie.retrievals.retrieve(request={
    "query": "Give detailed description of the video with timestamps of the events"
})

print(response)

scored_chunks=[ScoredChunk(text='{"video_description": "Okay, here\'s a detailed description of the video:\\n\\n**Overall Impression:**\\n\\nThe video focuses on a tense moment from a football (soccer) match. It starts with a game highlight and transitions into promotional content.\\n\\n**Visual Breakdown:**\\n\\n*   **0:00-0:06:** A game clip shows an intense moment near the goal.\\n    *   The camera focuses on the goal area in a packed stadium.\\n    *   The score is ARG 2-2 FRA at 107:55 in the game.\\n    *   A player shoots, the goalkeeper blocks but deflects the ball. Another player then shoots in the rebound attempt.\\n    *   There is audible cheering and excitement from the crowd.\\n\\n*   **0:07-0:15:** Advertisement/Promotional content:\\n    *   The background transitions to a deep blue colour.\\n    *   The following text overlays are displayed in white: \\"THE ACTION IS ONLY JUST KICKING OFF\\", \\"ONLY A\\", \\"FIFA\\", \\"STREAMING THE FREE STREAMING PLATFORM\\".\\n   

In [7]:
content = []
for chunk in response.scored_chunks:
    print(chunk.text)
    print("#"*100)
    content.append({
            **chunk.document_metadata,
            "text": chunk.text,
            "document_name": chunk.document_name,
            "start_time": chunk.metadata.get("start_time"),
            "end_time": chunk.metadata.get("end_time")
        })

{"video_description": "Okay, here's a detailed description of the video:\n\n**Overall Impression:**\n\nThe video focuses on a tense moment from a football (soccer) match. It starts with a game highlight and transitions into promotional content.\n\n**Visual Breakdown:**\n\n*   **0:00-0:06:** A game clip shows an intense moment near the goal.\n    *   The camera focuses on the goal area in a packed stadium.\n    *   The score is ARG 2-2 FRA at 107:55 in the game.\n    *   A player shoots, the goalkeeper blocks but deflects the ball. Another player then shoots in the rebound attempt.\n    *   There is audible cheering and excitement from the crowd.\n\n*   **0:07-0:15:** Advertisement/Promotional content:\n    *   The background transitions to a deep blue colour.\n    *   The following text overlays are displayed in white: \"THE ACTION IS ONLY JUST KICKING OFF\", \"ONLY A\", \"FIFA\", \"STREAMING THE FREE STREAMING PLATFORM\".\n    *   Flashes of popular football players are shown briefly:

## Video chunk creation

In [8]:
content[0]

{'text': '{"video_description": "Okay, here\'s a detailed description of the video:\\n\\n**Overall Impression:**\\n\\nThe video focuses on a tense moment from a football (soccer) match. It starts with a game highlight and transitions into promotional content.\\n\\n**Visual Breakdown:**\\n\\n*   **0:00-0:06:** A game clip shows an intense moment near the goal.\\n    *   The camera focuses on the goal area in a packed stadium.\\n    *   The score is ARG 2-2 FRA at 107:55 in the game.\\n    *   A player shoots, the goalkeeper blocks but deflects the ball. Another player then shoots in the rebound attempt.\\n    *   There is audible cheering and excitement from the crowd.\\n\\n*   **0:07-0:15:** Advertisement/Promotional content:\\n    *   The background transitions to a deep blue colour.\\n    *   The following text overlays are displayed in white: \\"THE ACTION IS ONLY JUST KICKING OFF\\", \\"ONLY A\\", \\"FIFA\\", \\"STREAMING THE FREE STREAMING PLATFORM\\".\\n    *   Flashes of popular

Let's see the above video clip to verify the video_description is correct

In [12]:
from moviepy import VideoFileClip

video = VideoFileClip("videos/messi-goals.mp4")
goal_clip = video.subclipped(content[0]['start_time'], content[0]['end_time'])
goal_clip.write_videofile("videos/goal_clip.mp4")

MoviePy - Building video videos/goal_clip.mp4.
MoviePy - Writing audio in goal_clipTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
MoviePy - Writing video videos/goal_clip.mp4



MoviePy - Done !
MoviePy - video ready videos/goal_clip.mp4


In [13]:
from IPython.display import HTML
from pathlib import Path

# View the created video chunk
video_path = Path('videos/goal_clip.mp4')
if video_path.exists():
    video_html = f'''
    <video width="640" height="480" controls>
        <source src="{video_path}" type="video/mp4">
        Your browser does not support the video tag.
    </video>
    '''
    display(HTML(video_html))
else:
    print("Video file not found!")